# Imports

In [ ]:
from __future__ import annotations
import warnings
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import compose, ensemble, impute, linear_model, metrics, model_selection, naive_bayes, neighbors, pipeline, preprocessing, svm, tree
from skopt import searchcv
from scipy import stats

warnings.simplefilter(action="ignore", category=UserWarning)

%matplotlib inline
pd.options.display.max_columns = None
plt.style.use("ggplot")

In [ ]:
df = pd.read_csv('wine-quality.csv')

# Data Preparation

## Check for duplicate rows

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()
df.shape

## Change spaces in column names to underscores

In [ ]:
df.columns = df.columns.str.replace(' ', '_')


## Check for missing values

In [ ]:
df.isna().sum()

# Exploratory Data Analysis

## Basic information about the dataset

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include="object")

In [ ]:
numeric_features: list[str] = df.select_dtypes(exclude="object").columns.array
categorial_features: list[str] = df.select_dtypes(include="object").columns.array

<hr />

## Distribution of each feature

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(20, 16), tight_layout=True)

for i, ax in enumerate(axes.flat):
    sns.kdeplot(df[numeric_features[i]], fill=True, ax=ax)

These histograms tell me that the distribution of the data for all of the skewed to the left. This means that I will need to do some data transformation on the fields I decide to keep, so that the data is more normally distributed.

<hr />

In [ ]:
fig, axes = plt.subplots(12, 1, figsize=(20, 16), tight_layout=True)

for i, ax in enumerate(axes.flat):
    sns.boxplot(x=df[numeric_features[i]], ax=ax)

These boxplots tell me that there are some outliers in the data. I will need to decide if I want to keep these outliers or not.

<hr />

## Handling Outliers

For each of the fields, I decided to do some trail and error to find out what percentile of the data (top and bottom percentile) I want to replace with the median value of the column, keeping the percentile value change to a minimum so as to not impute too many fields, but also to remove as many outliers as possible. Here are the outcomes of my trail and erroring

In [ ]:
trim_config: dict[str, tuple[int, int]] = {
    "fixed_acidity": (0.03, 0.92),
    "volatile_acidity": (0, 0.81),
    "citric_acid": (0.06, 0.95),
    "residual_sugar": (0, 0.93),
    "chlorides": (0, 0.92),
    "free_sulfur_dioxide": (0, 0.9),
    "total_sulfur_dioxide": (0, 0.95),
    "density": (0, 0.99),
    "pH": (0.01, 0.98),
    "sulphates": (0, 0.96),
    "alcohol": (0, 0.99),
    "quality": (0.01, 0.97)
}

for feature, iqr_range in trim_config.items():
    df[f"{feature}_trimmed"] = df[feature].clip(
        df[feature].quantile(iqr_range[0]),
        df[feature].quantile(iqr_range[1])
    )

In [ ]:
fig, axes = plt.subplots(12, 1, figsize=(20, 24), tight_layout=True)

for i, ax in enumerate(axes.flat):
    feature = numeric_features[i]
    sns.boxplot(
        pd.melt(df[[feature, f"{feature}_trimmed"]]),
        x="value",
        y="variable",
        ax=ax
    ).set(
        xlabel=None,
        ylabel=None
    )

### Overwriting the original columns with the trimmed columns

In [ ]:
for feature in numeric_features:
    df[feature] = df[f"{feature}_trimmed"]
    df: pd.DataFrame = df.drop(columns=[f"{feature}_trimmed"])

## Correlation of each feature

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

sns.heatmap(
    df[numeric_features].corr(),
    vmin=-1,
    vmax=1,
    annot=True,
    cmap="coolwarm",
    ax=ax,
)

This correlation heatmap tells me that the features have a moderate correlation with each other. This means that I will need to do some feature selection to reduce the number of features I use in my model.

<hr />

## Correlation of each feature with the target

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(20, 16), tight_layout=True)

for i, ax in enumerate(axes.flat):
    sns.lineplot(df, x="quality", y=numeric_features[i], ax=ax)

These graphs tell me that some of the features are more likely to affect the quality of the wine than others since the lines are more linear than the others, and can be used to predict the quality of the wine better. 

<hr />

# Feature Engineering & Model Building

## Normally Distributing the Data

I decided to transform each column individually so that they aren't skewed to either direction

In [ ]:
pd.DataFrame(
    [df[feature].skew() for feature in numeric_features],
    index=numeric_features,
    columns=["Skew"]
).T

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(20, 12), tight_layout=True)

for i, ax in enumerate(axes.flat):
    sns.kdeplot(df[numeric_features[i]], fill=True, ax=ax)

In [ ]:
for feature in numeric_features:
    df[f"{feature}_transformed"] = stats.boxcox(df[feature])[0]

In [ ]:
pd.DataFrame(
    [[df[feature].skew(), df[f"{feature}_transformed"].skew()] for feature in numeric_features],
    index=numeric_features,
    columns=["Skew before transformation", "Skew after transformation"]
).T

In [ ]:
fig, axes = plt.subplots(8, 3, figsize=(20, 24), tight_layout=True)

for i, ax in enumerate(axes.flat):
    # some magic to get the right feature name
    feature = numeric_features[i - ((i % 6 > 2) + i // 6) * 3]
    if i % 6 > 2:
        sns.kdeplot(df[f"{feature}_transformed"], fill=True, color="green", ax=ax)
    else:
        sns.kdeplot(df[feature], fill=True, ax=ax)

### Overwriting the original columns with the transformed columns

In [ ]:
for feature in numeric_features:
    df[feature] = df[f"{feature}_transformed"]
    df: pd.DataFrame = df.drop(columns=[f"{feature}_transformed"])

## Normalizing the data

I am using a pipeline with a standard scaler to normalize my numeric columns

In [ ]:
numeric_pipeline = pipeline.Pipeline([
    ("scaler", preprocessing.MinMaxScaler())
])

## One Hot Encoding

I am using a pipeline with a one hot encoder to one hot encode the categorical features

In [ ]:
categorical_pipeline = pipeline.Pipeline([
    ("onehot", preprocessing.OneHotEncoder(handle_unknown="ignore"))
])

## Classifying the target

I decided to split the output into 3 classes, 1 for low quality, 1 for medium quality, and 1 for high quality. This is because my output variable is not exactly continuous (since the value is either 3, 4, 5, 6, 7, 8 or 9). However if I use regression to solve this, I will have to use a continuous output variable, so I decided to use classification instead.

In [ ]:
sns.histplot(df["quality"], bins=10)

From the above histogram we can roughly split the data into 3 classes, 1 for low quality, 1 for medium quality, and 1 for high quality. I will use this to classify the data.

In [ ]:
df["quality_class"] = np.where(df["quality"] <= 4, "low", np.where(df["quality"] >= 7, "high", "medium"))

In [ ]:
sns.histplot(df["quality_class"])

In [ ]:
df = df.drop(columns=["quality"])

## First Draft of Model

I need to have at least one draft of the model before I can decide which features to keep, remove and scale for Feature Engineering. Becuase of this, I will run a simple RandomForestClassifier model with BayesSearchCV to find the best parameters for the model.

In [ ]:
running_bayes = True

The parameters here were found through a previous search, in the cases where I don't want to run bayes again since it can be very time consuming during testing

In [ ]:
bayes_optimal = {
    "max_depth": 5697,
    "min_samples_leaf": 1,
    "min_samples_split": 3,
    "n_estimators": 665
}

In [ ]:
rfc_pipeline = pipeline.Pipeline([
    ("preprocessor", compose._column_transformer.ColumnTransformer([
        ("numeric", numeric_pipeline, df[[feature for feature in numeric_features if feature != "quality"]].columns),
        ("categorical", categorical_pipeline, df[categorial_features].columns)
    ])),
    ("classifier", ensemble.RandomForestClassifier(
        **({} if running_bayes else bayes_optimal),
        random_state=hash("2100326D") % 2 ** 32,
        n_jobs=-1
    ))
])

In [ ]:
X = df.drop(columns=["quality_class"])
y = df["quality_class"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7, random_state=hash("2100326D") % 2 ** 32)
X_train: pd.DataFrame
X_test: pd.DataFrame
y_train: pd.Series
y_test: pd.Series

Now depending on whether I've run BayesSearchCV, I will either load the best parameters found or calculate them again

In [ ]:
if running_bayes:
    # Fix a bug in scikit-optimize
    np.int = int

    bs = searchcv.BayesSearchCV(
        rfc_pipeline,
        {
            "classifier__max_depth": (1, 10000),
            "classifier__min_samples_leaf": (1, 5),
            "classifier__min_samples_split": (2, 5),
            "classifier__n_estimators": (1, 1000)
        },
        n_iter=250,
        cv=5,
        n_jobs=-1,
        verbose=2
    )

    bs.fit(X_train, y_train)
    bayes_optimal = {
        "max_depth": bs.best_params_["classifier__max_depth"],
        "min_samples_leaf": bs.best_params_["classifier__min_samples_leaf"],
        "min_samples_split": bs.best_params_["classifier__min_samples_split"],
        "n_estimators": bs.best_params_["classifier__n_estimators"]
    }

    print("Best params:", bayes_optimal)
    print("Best score:", bs.best_score_)
    print("Train score:", bs.score(X_train, y_train))
    print("Test score:", bs.score(X_test, y_test))
else:
    rfc_pipeline.fit(X_train, y_train)

    print("Train score:", rfc_pipeline.score(X_train, y_train))
    print("Test score:", rfc_pipeline.score(X_test, y_test))

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

rfc_pipeline: pipeline.Pipeline = bs.best_estimator_ if running_bayes else rfc_pipeline
_transformer: compose._column_transformer.ColumnTransformer = rfc_pipeline.steps[0][1]
_classifier: ensemble.RandomForestClassifier = rfc_pipeline.steps[1][1]
_categorical_pipeline: pipeline.Pipeline = _transformer.transformers_[1][1]
_onehot: preprocessing.OneHotEncoder = _categorical_pipeline.steps[0][1]

importances = pd.DataFrame(
    _classifier.feature_importances_,
    index=list(X_train.drop(columns=["type"]).columns) + _onehot.get_feature_names_out().tolist(),
    columns=["Importance"]
).sort_values(by="Importance", ascending=True).T

sns.barplot(
    data=importances.T[::-1],
    x="Importance",
    y=importances.columns[::-1],
    ax=ax
)

## Feature Selection

Before I do feature selection, here is a reusable function to fit the model and return it's scores. It modifies the pipelines to work when some columns are missing

In [ ]:
def fit_with_piped_features(piped_features: list[str], running_combinations = False) -> tuple[int, int]:
    rfc_pipeline = pipeline.Pipeline([
        ("preprocessor", compose._column_transformer.ColumnTransformer([
            ("numeric", numeric_pipeline,
                df[[feature for feature in piped_features if feature != "type"]].columns
            ),
            ("categorical", categorical_pipeline,
                df[["type"]].columns if "type" in piped_features else []
            )
        ])),
        ("classifier", ensemble.RandomForestClassifier(
            **({} if running_combinations else bayes_optimal),
            random_state=hash("2100326D") % 2 ** 32,
            n_jobs=-1
        ))
    ])
    
    rfc_pipeline.fit(X_train[piped_features], y_train)
    train_score = rfc_pipeline.score(X_train[piped_features], y_train)
    test_score = rfc_pipeline.score(X_test[piped_features], y_test)

    return train_score, test_score



First I decided to try to fit the model slowly removing each feature in the order of feature importance ascending

In [ ]:
piped_features = (["type"] + list(numeric_features[:-1]))

for i in range(len(piped_features)):
    features = piped_features[i:]
    train_score, test_score = fit_with_piped_features(features)

    print(f"Train score {features}:", train_score)
    print(f"Test score {features}:", test_score)
    print()

The highest score was `0.814` which was ok but I wanted to know if there are any better scores with other combinations of features I put into the model

<hr />

I'm going to run fit the model with all possible combination of the feature list to find which combination gives me the best score for the model.

In [ ]:
running_combinations = True

These values are the optimal values found in the previous run of this notebook.

In [ ]:
combinations_optimal = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates']
scores_optimal = (0, 0)

Now depending on whether I've run the combinations already, I will either load the previous data or calculate it again

In [ ]:
piped_features = (["type"] + list(numeric_features[:-1]))

if running_combinations:
    for i in range(1, len(piped_features)):
        for features in itertools.combinations(piped_features, i):
            features = list(features)
            train_score, test_score = fit_with_piped_features(features)

            print(f"Train score {features}:", train_score)
            print(f"Test score {features}:", test_score)

            if test_score > scores_optimal[1]:
                print("Highscore!")
                combinations_optimal = features
                scores_optimal = (train_score, test_score)

            print()
    print()
else:
    scores_optimal = fit_with_piped_features(combinations_optimal, running_combinations)

print("Highest train score:", scores_optimal[0])
print("Highest test score:", scores_optimal[1])
print("Included Features:", combinations_optimal)
print("Excluded Features:", [feature for feature in (["type"] + list(numeric_features[:-1])) if feature not in combinations_optimal])

It's funny how I got the highest model score when the feature with the highest importance was left out. Although this isn't a good practice, I will follow what the output of the code tells me about what the optimal features

<hr />

In [ ]:
numeric_features = [feature for feature in numeric_features if feature not in ["chlorides", "alcohol"]]
del categorial_features

X_train = X_train.drop(columns=["type", "chlorides", "alcohol"])
y_train = y_train.drop(columns=["type", "chlorides", "alcohol"])

## Final Model

Now that I am done with Feature Engineering, I can build the final version of the model. I will also run bayes again to find the best parameters for my model since my dataset changed a bit.

In [ ]:
running_bayes = True

The parameters here were found through a previous search, in the cases where I don't want to run bayes again since it can be very time consuming during testing

In [ ]:
bayes_optimal = {
    "max_depth": 6211,
    "min_samples_leaf": 1,
    "min_samples_split": 4,
    "n_estimators": 129
}

In [ ]:
rfc_pipeline = pipeline.Pipeline([
    ("preprocessor", compose._column_transformer.ColumnTransformer([
        ("numeric", numeric_pipeline, df[[feature for feature in numeric_features if feature != "quality"]].columns),
    ])),
    ("classifier", ensemble.RandomForestClassifier(
        **({} if running_bayes else bayes_optimal),
        random_state=hash("2100326D") % 2 ** 32,
        n_jobs=-1
    ))
])

Now depending on whether I've run BayesSearchCV already, I will either load the best parameters found or calculate them again

In [ ]:
if running_bayes:
    # Fix a bug in scikit-optimize
    np.int = int

    bs = searchcv.BayesSearchCV(
        rfc_pipeline,
        {
            "classifier__max_depth": (1, 10000),
            "classifier__min_samples_leaf": (1, 5),
            "classifier__min_samples_split": (2, 5),
            "classifier__n_estimators": (1, 1000)
        },
        n_iter=250,
        cv=5,
        n_jobs=-1,
        verbose=2
    )

    bs.fit(X_train, y_train)
    bayes_optimal = {
        "max_depth": bs.best_params_["classifier__max_depth"],
        "min_samples_leaf": bs.best_params_["classifier__min_samples_leaf"],
        "min_samples_split": bs.best_params_["classifier__min_samples_split"],
        "n_estimators": bs.best_params_["classifier__n_estimators"]
    }

    print("Best params:", bs.best_params_)
    print("Best score:", bs.best_score_)
    print("Train score:", bs.score(X_train, y_train))
    print("Test score:", bs.score(X_test, y_test))
else:
    rfc_pipeline.fit(X_train, y_train)

    print("Train score:", rfc_pipeline.score(X_train, y_train))
    print("Test score:", rfc_pipeline.score(X_test, y_test))